In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from collections import Counter
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer
import sys
import unicodedata
import string
import operator
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy import sparse
from scipy.spatial.distance import cosine
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
stdout = sys.stdout
reload(sys)  
sys.setdefaultencoding('utf8')
sys.stdout = stdout

In [2]:
import jsonlines
import numpy as np
import cPickle as pickle
import os

class Preprocessor:
    def __init__(self, dataDir = './'):
        self.data = []
        self.dataDir = dataDir 

    def isSampleValid(self, sample):
        return sample['gold_label'] != '-'
    
    def getDataFromJSONL(self, filename):
        numInvalid = 0
        with open(filename) as fp:
            reader = jsonlines.Reader(fp)
            for obj in reader.iter(type=dict, skip_invalid=True):
                if self.isSampleValid(obj):
                    sample = {}
                    sample['gold_label'] = obj['gold_label']
                    sample['sentence1'] = obj['sentence1']
                    sample['sentence2'] = obj['sentence2']
                    self.data.append(sample)
                else:
                    numInvalid+=1
        print len(self.data)
        print numInvalid
    
    

In [3]:
pp = Preprocessor()
pp.getDataFromJSONL('./snli_1.0_train.jsonl')

549367
785


In [4]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
BLEU_Score_List = []
for j in range(len(pp.data)):
    weights = [1./4., 1./4., 1./4., 1./4.]
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    reference = [[i for i in sentence1.lower().split() if i not in stop]]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    candidate = [i for i in sentence2.lower().split() if i not in stop]
    length = min([len(reference[0]), len(candidate)]) 
    if length == 0:
        BLEU_Score_List.append(0)
    else:
        if length < 4:
            weights = ( 1. / length ,) * length
        score = sentence_bleu(reference, candidate, weights)
        BLEU_Score_List.append(score)      

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/seasnake/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
%store BLEU_Score_List > BLEU_Score_List.txt

Writing 'BLEU_Score_List' (list) to file 'BLEU_Score_List.txt'.


In [4]:
BLEU_Score_List = eval(open("BLEU_Score_List.txt").read())  
BLEU_Score_Array = np.array(BLEU_Score_List)

In [5]:
BLEU_Score_Array = BLEU_Score_Array[:, np.newaxis]

In [8]:
Target_List = [pp.data[j]['gold_label'] for j in range(len(pp.data))]

In [9]:
%store Target_List > Target_List.txt

Writing 'Target_List' (list) to file 'Target_List.txt'.


In [6]:
Target_List = eval(open("Target_List.txt").read())  
Target_Array = np.array(Target_List)

In [8]:
print Target_Array.shape

(549367,)


In [9]:
print BLEU_Score_Array.shape

(549367, 1)


In [10]:
tt = Preprocessor()
tt.getDataFromJSONL('./snli_1.0_test.jsonl')

9824
176


In [42]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
BLEU_Score_List_Test = []
for j in range(len(tt.data)):
    weights = [1./4., 1./4., 1./4., 1./4.]
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    reference = [[i for i in sentence1.lower().split() if i not in stop]]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    candidate = [i for i in sentence2.lower().split() if i not in stop]
    length = min([len(reference[0]), len(candidate)]) 
    if length == 0:
        BLEU_Score_List_Test.append(0)
    else:
        if length < 4:
            weights = ( 1. / length ,) * length
        score = sentence_bleu(reference, candidate, weights)
        BLEU_Score_List_Test.append(score)      

In [44]:
%store BLEU_Score_List_Test > BLEU_Score_List_Test.txt

Writing 'BLEU_Score_List_Test' (list) to file 'BLEU_Score_List_Test.txt'.


In [11]:
BLEU_Score_List_Test = eval(open("BLEU_Score_List_Test.txt").read())  
BLEU_Score_Array_Test = np.array(BLEU_Score_List_Test)

In [12]:
BLEU_Score_Array_Test = BLEU_Score_Array_Test[:, np.newaxis]

In [47]:
Target_List_Test = [tt.data[j]['gold_label'] for j in range(len(tt.data))]

In [48]:
%store Target_List_Test > Target_List_Test.txt

Writing 'Target_List_Test' (list) to file 'Target_List_Test.txt'.


In [13]:
Target_List_Test = eval(open("Target_List_Test.txt").read())
Target_Array_Test = np.array(Target_List_Test)

In [15]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_train = gnb.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array)
y_pred_list_train = y_pred_train.tolist()
%store y_pred_list_train > y_pred_list_train.txt

Writing 'y_pred_list_train' (list) to file 'y_pred_list_train.txt'.


In [18]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array.shape[0],(Target_Array != y_pred_train).sum()))

Number of mislabeled points out of a total 549367 points : 331625


In [19]:
print 549367 - 331625

217742


In [20]:
print 217742./549367.

0.396350709089


In [13]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list = y_pred.tolist()
%store y_pred_list > y_pred_list.txt

Writing 'y_pred_list' (list) to file 'y_pred_list.txt'.


In [14]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred).sum()))

Number of mislabeled points out of a total 9824 points : 5957


In [51]:
print 9824 - 5957

3867


In [52]:
print 3867./9824.

0.393627850163


In [15]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
y_pred_mnb = mnb.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list_mnb = y_pred_mnb.tolist()
%store y_pred_list_mnb > y_pred_list_mnb.txt

Writing 'y_pred_list_mnb' (list) to file 'y_pred_list_mnb.txt'.


In [16]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred_mnb).sum()))

Number of mislabeled points out of a total 9824 points : 6456


In [27]:
print 9824 - 6456

3368


In [28]:
print 3368./9824.

0.342833876221


In [21]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_svm_train = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array)
y_pred_list_svm_train = y_pred_svm_train.tolist()
%store y_pred_list_svm_train > y_pred_list_svm_train.txt

Writing 'y_pred_list_svm_train' (list) to file 'y_pred_list_svm_train.txt'.


In [22]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array.shape[0],(Target_Array != y_pred_svm_train).sum()))

Number of mislabeled points out of a total 549367 points : 361681


In [23]:
print 549367 - 361681

187686


In [24]:
print 187686./549367.

0.34164046985


In [57]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_svm = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list_svm = y_pred_svm.tolist()
%store y_pred_list_svm > y_pred_list_svm.txt

Writing 'y_pred_list_svm' (list) to file 'y_pred_list_svm.txt'.


In [58]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred_svm).sum()))

Number of mislabeled points out of a total 9824 points : 6587


In [59]:
print 9824 - 6587

3237


In [60]:
print 3237./9824.

0.329499185668


In [25]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_lr_train = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array)
y_pred_list_lr_train = y_pred_lr_train.tolist()
%store y_pred_list_lr_train > y_pred_list_lr_train.txt

Writing 'y_pred_list_lr_train' (list) to file 'y_pred_list_lr_train.txt'.


In [26]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array.shape[0],(Target_Array != y_pred_lr_train).sum()))

Number of mislabeled points out of a total 549367 points : 344628


In [27]:
print 549367 - 344628

204739


In [28]:
print 204739./549367.

0.372681649972


In [13]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_lr = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list_lr = y_pred_lr.tolist()
%store y_pred_list_lr > y_pred_list_lr.txt

Writing 'y_pred_list_lr' (list) to file 'y_pred_list_lr.txt'.


In [14]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred_lr).sum()))

Number of mislabeled points out of a total 9824 points : 6198


In [15]:
print 9824 - 6198

3626


In [16]:
print 3626./9824.

0.369096091205


In [61]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred))

             precision    recall  f1-score   support

contradiction       0.38      0.44      0.41      3237
 entailment       0.39      0.35      0.37      3368
    neutral       0.41      0.39      0.40      3219

avg / total       0.39      0.39      0.39      9824



In [62]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_mnb))

             precision    recall  f1-score   support

contradiction       0.00      0.00      0.00      3237
 entailment       0.34      1.00      0.51      3368
    neutral       0.00      0.00      0.00      3219

avg / total       0.12      0.34      0.18      9824



In [63]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_svm))

             precision    recall  f1-score   support

contradiction       0.33      1.00      0.50      3237
 entailment       0.00      0.00      0.00      3368
    neutral       0.00      0.00      0.00      3219

avg / total       0.11      0.33      0.16      9824



In [17]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_lr))

             precision    recall  f1-score   support

contradiction       0.36      0.63      0.45      3237
 entailment       0.48      0.01      0.02      3368
    neutral       0.39      0.48      0.43      3219

avg / total       0.41      0.37      0.30      9824



In [64]:
metrics.confusion_matrix(Target_Array_Test, y_pred)


array([[1434,  919,  884],
       [1253, 1192,  923],
       [1062,  916, 1241]])

In [18]:
metrics.confusion_matrix(Target_Array_Test, y_pred_lr)

array([[2034,   22, 1181],
       [2044,   39, 1285],
       [1645,   21, 1553]])

In [68]:
Length_Difference_List = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [i for i in sentence1.lower().split()]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [i for i in sentence2.lower().split()]
    length_diff = len(premise)-len(hypothesis)
    Length_Difference_List.append(length_diff)    

In [69]:
%store Length_Difference_List > Length_Difference_List.txt

Writing 'Length_Difference_List' (list) to file 'Length_Difference_List.txt'.


In [29]:
Length_Difference_List = eval(open("Length_Difference_List.txt").read())  
Length_Difference_Array = np.array(Length_Difference_List)

In [30]:
Length_Difference_Array = Length_Difference_Array[:, np.newaxis]

In [31]:
Two_Features = np.concatenate((BLEU_Score_Array, Length_Difference_Array), axis=1)

In [32]:
print Two_Features.shape

(549367, 2)


In [77]:
Length_Difference_List_Test = []
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [i for i in sentence1.lower().split()]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [i for i in sentence2.lower().split()]
    length_diff = len(premise)-len(hypothesis)
    Length_Difference_List_Test.append(length_diff)    

In [78]:
%store Length_Difference_List_Test > Length_Difference_List_Test.txt

Writing 'Length_Difference_List_Test' (list) to file 'Length_Difference_List_Test.txt'.


In [33]:
Length_Difference_List_Test = eval(open("Length_Difference_List_Test.txt").read())
Length_Difference_Array_Test = np.array(Length_Difference_List_Test)

In [34]:
Length_Difference_Array_Test = Length_Difference_Array_Test[:, np.newaxis]

In [35]:
Two_Features_Test = np.concatenate((BLEU_Score_Array_Test, Length_Difference_Array_Test), axis=1)

In [36]:
print Two_Features_Test.shape

(9824, 2)


In [37]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_2_train = gnb.fit(Two_Features, Target_Array).predict(Two_Features)
y_pred_2_list_train = y_pred_2_train.tolist()
%store y_pred_2_list_train > y_pred_2_list_train.txt

Writing 'y_pred_2_list_train' (list) to file 'y_pred_2_list_train.txt'.


In [38]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features.shape[0],(Target_Array != y_pred_2_train).sum()))

Number of mislabeled points out of a total 549367 points : 329733


In [39]:
print 549367 - 329733

219634


In [40]:
print 219634./549367.

0.399794672778


In [83]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_2 = gnb.fit(Two_Features, Target_Array).predict(Two_Features_Test)
y_pred_2_list = y_pred_2.tolist()
%store y_pred_2_list > y_pred_2_list.txt

Writing 'y_pred_2_list' (list) to file 'y_pred_2_list.txt'.


In [84]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_2).sum()))

Number of mislabeled points out of a total 9824 points : 5897


In [86]:
print 9824 - 5897

3927


In [87]:
print 3927./9824.

0.39973534202


In [88]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_2))

             precision    recall  f1-score   support

contradiction       0.42      0.21      0.28      3237
 entailment       0.39      0.59      0.47      3368
    neutral       0.40      0.40      0.40      3219

avg / total       0.41      0.40      0.38      9824



In [89]:
metrics.confusion_matrix(Target_Array_Test, y_pred_2)


array([[ 677, 1624,  936],
       [ 434, 1977,  957],
       [ 490, 1456, 1273]])

In [41]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_2_svm_train = clf.fit(Two_Features, Target_Array).predict(Two_Features)
y_pred_list_svm_2_train = y_pred_2_svm_train.tolist()
%store y_pred_list_svm_2_train > y_pred_list_svm_2_train.txt

Writing 'y_pred_list_svm_2_train' (list) to file 'y_pred_list_svm_2_train.txt'.


In [42]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features.shape[0],(Target_Array != y_pred_2_svm_train).sum()))

Number of mislabeled points out of a total 549367 points : 336877


In [43]:
print 549367 - 336877

212490


In [44]:
print 212490./549367.

0.386790615381


In [91]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_2_svm = clf.fit(Two_Features, Target_Array).predict(Two_Features_Test)
y_pred_list_svm_2 = y_pred_2_svm.tolist()
%store y_pred_list_svm_2 > y_pred_list_svm_2.txt

Writing 'y_pred_list_svm_2' (list) to file 'y_pred_list_svm_2.txt'.


In [92]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_2_svm).sum()))

Number of mislabeled points out of a total 9824 points : 6005


In [93]:
print 9824 - 6005

3819


In [94]:
print 3819./9824.

0.388741856678


In [95]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_2_svm))

             precision    recall  f1-score   support

contradiction       0.44      0.04      0.08      3237
 entailment       0.37      0.89      0.52      3368
    neutral       0.49      0.21      0.29      3219

avg / total       0.43      0.39      0.30      9824



In [96]:
metrics.confusion_matrix(Target_Array_Test, y_pred_2_svm)


array([[ 136, 2688,  413],
       [  71, 3013,  284],
       [ 105, 2444,  670]])

In [45]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_2_lr_train = clf.fit(Two_Features, Target_Array).predict(Two_Features)
y_pred_list_lr_2_train = y_pred_2_lr_train.tolist()
%store y_pred_list_lr_2_train > y_pred_list_lr_2_train.txt

Writing 'y_pred_list_lr_2_train' (list) to file 'y_pred_list_lr_2_train.txt'.


In [46]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features.shape[0],(Target_Array != y_pred_2_lr_train).sum()))

Number of mislabeled points out of a total 549367 points : 337283


In [47]:
print 549367 - 337283

212084


In [48]:
print 212084./549367.

0.386051583004


In [27]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_2_lr = clf.fit(Two_Features, Target_Array).predict(Two_Features_Test)
y_pred_list_lr_2 = y_pred_2_lr.tolist()
%store y_pred_list_lr_2 > y_pred_list_lr_2.txt

Writing 'y_pred_list_lr_2' (list) to file 'y_pred_list_lr_2.txt'.


In [28]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_2_lr).sum()))

Number of mislabeled points out of a total 9824 points : 6029


In [29]:
print 9824 - 6029

3795


In [30]:
print 3795./9824.

0.386298859935


In [31]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_2_lr))

             precision    recall  f1-score   support

contradiction       0.38      0.33      0.36      3237
 entailment       0.39      0.40      0.39      3368
    neutral       0.39      0.43      0.41      3219

avg / total       0.39      0.39      0.39      9824



In [132]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Overlap_Percent = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    lengtharray = np.array([len(premise), len(hypothesis)])
    length = np.min(lengtharray)
    indx = np.argmin(lengtharray)
    if length == 0:
        Overlap_Percent.append(0)
    else:
        count = 0
        if indx == 0:
            for j in range(length):
                if premise[j] in hypothesis:
                    count = count+1
        else:
            for j in range(length):
                if hypothesis[j] in premise:
                    count = count+1
        Overlap_Percent.append(count/float(length))                        

In [135]:
%store Overlap_Percent > Overlap_Percent.txt

Writing 'Overlap_Percent' (list) to file 'Overlap_Percent.txt'.


In [49]:
Overlap_Percent = eval(open("Overlap_Percent.txt").read())
Overlap_Percent_Array = np.array(Overlap_Percent)

In [50]:
Overlap_Percent_Array = Overlap_Percent_Array[:, np.newaxis]

In [138]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Overlap_Percent_Test = []
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    lengtharray = np.array([len(premise), len(hypothesis)])
    length = np.min(lengtharray)
    indx = np.argmin(lengtharray)
    if length == 0:
        Overlap_Percent_Test.append(0)
    else:
        count = 0
        if indx == 0:
            for j in range(length):
                if premise[j] in hypothesis:
                    count = count+1
        else:
            for j in range(length):
                if hypothesis[j] in premise:
                    count = count+1
        Overlap_Percent_Test.append(count/float(length))  

In [139]:
%store Overlap_Percent_Test > Overlap_Percent_Test.txt

Writing 'Overlap_Percent_Test' (list) to file 'Overlap_Percent_Test.txt'.


In [51]:
Overlap_Percent_Test = eval(open("Overlap_Percent_Test.txt").read())
Overlap_Percent_Array_Test = np.array(Overlap_Percent_Test)

In [52]:
Overlap_Percent_Array_Test = Overlap_Percent_Array_Test[:, np.newaxis]

In [53]:
Three_Features = np.concatenate((Two_Features, Overlap_Percent_Array), axis=1)

In [54]:
print Three_Features.shape

(549367, 3)


In [55]:
Three_Features_Test = np.concatenate((Two_Features_Test, Overlap_Percent_Array_Test), axis=1)

In [56]:
print Three_Features_Test.shape

(9824, 3)


In [57]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_3_train = gnb.fit(Three_Features, Target_Array).predict(Three_Features)
y_pred_3_list_train = y_pred_3_train.tolist()
%store y_pred_3_list_train > y_pred_3_list_train.txt

Writing 'y_pred_3_list_train' (list) to file 'y_pred_3_list_train.txt'.


In [58]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features.shape[0],(Target_Array != y_pred_3_train).sum()))

Number of mislabeled points out of a total 549367 points : 293646


In [59]:
print 549367-293646

255721


In [60]:
print 255721./549367.

0.465483001345


In [146]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_3 = gnb.fit(Three_Features, Target_Array).predict(Three_Features_Test)
y_pred_3_list = y_pred_3.tolist()
%store y_pred_3_list > y_pred_3_list.txt

Writing 'y_pred_3_list' (list) to file 'y_pred_3_list.txt'.


In [147]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features_Test.shape[0],(Target_Array_Test != y_pred_3).sum()))

Number of mislabeled points out of a total 9824 points : 5174


In [99]:
print 9824 - 5174

4650


In [100]:
print 4650./9824.

0.473330618893


In [101]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3))

             precision    recall  f1-score   support

contradiction       0.43      0.54      0.48      3237
 entailment       0.55      0.54      0.55      3368
    neutral       0.43      0.34      0.38      3219

avg / total       0.47      0.47      0.47      9824



In [102]:
metrics.confusion_matrix(Target_Array_Test, y_pred_3)


array([[1735,  680,  822],
       [ 937, 1823,  608],
       [1340,  787, 1092]])

In [61]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_3_svm_train = clf.fit(Three_Features, Target_Array).predict(Three_Features)
y_pred_list_svm_3_train = y_pred_3_svm_train.tolist()
%store y_pred_list_svm_3_train > y_pred_list_svm_3_train.txt

Writing 'y_pred_list_svm_3_train' (list) to file 'y_pred_list_svm_3_train.txt'.


In [62]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features.shape[0],(Target_Array != y_pred_3_svm_train).sum()))

Number of mislabeled points out of a total 549367 points : 314507


In [63]:
print 549367 - 314507

234860


In [64]:
print 234860./549367.

0.427510207202


In [162]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_3_svm = clf.fit(Three_Features, Target_Array).predict(Three_Features_Test)
y_pred_list_svm_3 = y_pred_3_svm.tolist()
%store y_pred_list_svm_3 > y_pred_list_svm_3.txt

Writing 'y_pred_list_svm_3' (list) to file 'y_pred_list_svm_3.txt'.


In [163]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features_Test.shape[0],(Target_Array_Test != y_pred_3_svm).sum()))

Number of mislabeled points out of a total 9824 points : 5304


In [164]:
print 9824 - 5304

4520


In [165]:
print 4520./9824.

0.46009771987


In [166]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3_svm))

             precision    recall  f1-score   support

contradiction       0.42      0.63      0.51      3237
 entailment       0.49      0.64      0.56      3368
    neutral       0.49      0.10      0.17      3219

avg / total       0.47      0.46      0.41      9824



In [167]:
metrics.confusion_matrix(Target_Array_Test, y_pred_3_svm)


array([[2030,  995,  212],
       [1083, 2167,  118],
       [1667, 1229,  323]])

In [65]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_3_lr_train = clf.fit(Three_Features, Target_Array).predict(Three_Features)
y_pred_list_lr_3_train = y_pred_3_lr_train.tolist()
%store y_pred_list_lr_3_train > y_pred_list_lr_3_train.txt

Writing 'y_pred_list_lr_3_train' (list) to file 'y_pred_list_lr_3_train.txt'.


In [66]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features.shape[0],(Target_Array != y_pred_3_lr_train).sum()))

Number of mislabeled points out of a total 549367 points : 297395


In [67]:
print 549367 - 297395

251972


In [68]:
print 251972./549367.

0.458658783655


In [49]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_3_lr = clf.fit(Three_Features, Target_Array).predict(Three_Features_Test)
y_pred_list_lr_3 = y_pred_3_lr.tolist()
%store y_pred_list_lr_3 > y_pred_list_lr_3.txt

Writing 'y_pred_list_lr_3' (list) to file 'y_pred_list_lr_3.txt'.


In [50]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features_Test.shape[0],(Target_Array_Test != y_pred_3_lr).sum()))

Number of mislabeled points out of a total 9824 points : 5253


In [51]:
print 9824 - 5253

4571


In [52]:
print 4571./9824.

0.465289087948


In [53]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3_lr))

             precision    recall  f1-score   support

contradiction       0.45      0.48      0.46      3237
 entailment       0.50      0.62      0.56      3368
    neutral       0.42      0.29      0.34      3219

avg / total       0.46      0.47      0.46      9824



In [54]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3_lr))

             precision    recall  f1-score   support

contradiction       0.45      0.48      0.46      3237
 entailment       0.50      0.62      0.56      3368
    neutral       0.42      0.29      0.34      3219

avg / total       0.46      0.47      0.46      9824



In [31]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')

In [32]:
sid = SentimentIntensityAnalyzer()

In [50]:
sentence1 = pp.data[2]['sentence1']

In [51]:
ss = sid.polarity_scores(sentence1)

In [52]:
print ss

{'neg': 0.307, 'neu': 0.693, 'pos': 0.0, 'compound': -0.4767}


In [53]:
print sentence1

A person on a horse jumps over a broken down airplane.


In [54]:
sentence2 = pp.data[2]['sentence2']

In [55]:
ss = sid.polarity_scores(sentence2)

In [56]:
print ss

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [57]:
print sentence2

A person is outdoors, on a horse.


Counter({u' ': 6, u'o': 6, u's': 4, u'r': 3, u'e': 2, u'n': 2, u'A': 1, u'd': 1, u'i': 1, u'h': 1, u',': 1, u'p': 1, u'u': 1, u't': 1, u'a': 1, u'.': 1})


In [64]:
Premise_Sentiment = []
Hypothesis_Sentiment = []
Match = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    premise_ss = sid.polarity_scores(sentence1)
    Premise_Sentiment.append(premise_ss['compound'])
    sentence2 = pp.data[j]['sentence2']
    hypothesis_ss = sid.polarity_scores(sentence2)
    Hypothesis_Sentiment.append(hypothesis_ss['compound'])
    if premise_ss['compound'] >= 0 and hypothesis_ss['compound'] >= 0:
        Match.append(1)
    elif premise_ss['compound'] >=0 and hypothesis_ss['compound'] < 0:
        Match.append(-1)
    elif premise_ss['compound'] < 0 and hypothesis_ss['compound'] >= 0:
        Match.append(-1)
    else:
        Match.append(1)
%store Premise_Sentiment > Premise_Sentiment.txt
%store Hypothesis_Sentiment > Hypothesis_Sentiment.txt
%store Match > Match.txt

Writing 'Premise_Sentiment' (list) to file 'Premise_Sentiment.txt'.
Writing 'Hypothesis_Sentiment' (list) to file 'Hypothesis_Sentiment.txt'.
Writing 'Match' (list) to file 'Match.txt'.


In [65]:
Premise_Sentiment = eval(open("Premise_Sentiment.txt").read())
Premise_Sentiment_Array = np.array(Premise_Sentiment)
Hypothesis_Sentiment = eval(open("Hypothesis_Sentiment.txt").read())
Hypothesis_Sentiment_Array = np.array(Hypothesis_Sentiment)
Match = eval(open("Match.txt").read())
Match_Array = np.array(Match)

In [66]:
Premise_Sentiment_Array = Premise_Sentiment_Array[:, np.newaxis]

In [67]:
Hypothesis_Sentiment_Array = Hypothesis_Sentiment_Array[:, np.newaxis]

In [68]:
Match_Array = Match_Array[:, np.newaxis]

In [69]:
Six_Features = np.concatenate((Three_Features, Premise_Sentiment_Array, Hypothesis_Sentiment_Array, Match_Array), axis=1)

In [70]:
print Six_Features.shape

(549367, 6)


In [71]:
Premise_Sentiment_Test = []
Hypothesis_Sentiment_Test = []
Match_Test = []
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    premise_ss = sid.polarity_scores(sentence1)
    Premise_Sentiment_Test.append(premise_ss['compound'])
    sentence2 = tt.data[j]['sentence2']
    hypothesis_ss = sid.polarity_scores(sentence2)
    Hypothesis_Sentiment_Test.append(hypothesis_ss['compound'])
    if premise_ss['compound'] >= 0 and hypothesis_ss['compound'] >= 0:
        Match_Test.append(1)
    elif premise_ss['compound'] >=0 and hypothesis_ss['compound'] < 0:
        Match_Test.append(-1)
    elif premise_ss['compound'] < 0 and hypothesis_ss['compound'] >= 0:
        Match_Test.append(-1)
    else:
        Match_Test.append(1)
%store Premise_Sentiment_Test > Premise_Sentiment_Test.txt
%store Hypothesis_Sentiment_Test > Hypothesis_Sentiment_Test.txt
%store Match_Test > Match_Test.txt

Writing 'Premise_Sentiment_Test' (list) to file 'Premise_Sentiment_Test.txt'.
Writing 'Hypothesis_Sentiment_Test' (list) to file 'Hypothesis_Sentiment_Test.txt'.
Writing 'Match_Test' (list) to file 'Match_Test.txt'.


In [72]:
Premise_Sentiment_Test = eval(open("Premise_Sentiment_Test.txt").read())
Premise_Sentiment_Array_Test = np.array(Premise_Sentiment_Test)
Hypothesis_Sentiment_Test = eval(open("Hypothesis_Sentiment_Test.txt").read())
Hypothesis_Sentiment_Array_Test = np.array(Hypothesis_Sentiment_Test)
Match_Test = eval(open("Match_Test.txt").read())
Match_Array_Test = np.array(Match_Test)

In [73]:
Premise_Sentiment_Array_Test = Premise_Sentiment_Array_Test[:, np.newaxis]
Hypothesis_Sentiment_Array_Test = Hypothesis_Sentiment_Array_Test[:, np.newaxis]
Match_Array_Test = Match_Array_Test[:, np.newaxis]

In [74]:
Six_Features_Test = np.concatenate((Three_Features_Test, Premise_Sentiment_Array_Test, Hypothesis_Sentiment_Array_Test, Match_Array_Test), axis=1)

In [75]:
print Six_Features_Test.shape

(9824, 6)


In [76]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_6 = gnb.fit(Six_Features, Target_Array).predict(Six_Features_Test)
y_pred_6_list = y_pred_6.tolist()
%store y_pred_6_list > y_pred_6_list.txt

Writing 'y_pred_6_list' (list) to file 'y_pred_6_list.txt'.


In [79]:
print("Number of mislabeled points out of a total %d points : %d" % (Six_Features_Test.shape[0],(Target_Array_Test != y_pred_6).sum()))

Number of mislabeled points out of a total 9824 points : 5309


In [78]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_6_svm = clf.fit(Six_Features, Target_Array).predict(Six_Features_Test)
y_pred_list_svm_6 = y_pred_6_svm.tolist()
%store y_pred_list_svm_6 > y_pred_list_svm_6.txt

Writing 'y_pred_list_svm_6' (list) to file 'y_pred_list_svm_6.txt'.


In [80]:
print("Number of mislabeled points out of a total %d points : %d" % (Six_Features_Test.shape[0],(Target_Array_Test != y_pred_6_svm).sum()))

Number of mislabeled points out of a total 9824 points : 5115


In [81]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_6_lr = clf.fit(Six_Features, Target_Array).predict(Six_Features_Test)
y_pred_list_lr_6 = y_pred_6_lr.tolist()
%store y_pred_list_lr_6 > y_pred_list_lr_6.txt

Writing 'y_pred_list_lr_6' (list) to file 'y_pred_list_lr_6.txt'.


In [82]:
print("Number of mislabeled points out of a total %d points : %d" % (Six_Features_Test.shape[0],(Target_Array_Test != y_pred_6_lr).sum()))

Number of mislabeled points out of a total 9824 points : 5092


In [83]:
print 9824 - 5092

4732


In [84]:
print 4732./9824.

0.48167752443


In [85]:
Five_Features = np.concatenate((Three_Features, Hypothesis_Sentiment_Array, Match_Array), axis=1)

In [86]:
Five_Features_Test = np.concatenate((Three_Features_Test, Hypothesis_Sentiment_Array_Test, Match_Array_Test), axis=1)

In [87]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_5 = gnb.fit(Five_Features, Target_Array).predict(Five_Features_Test)
y_pred_5_list = y_pred_5.tolist()
%store y_pred_5_list > y_pred_5_list.txt

Writing 'y_pred_5_list' (list) to file 'y_pred_5_list.txt'.


In [88]:
print("Number of mislabeled points out of a total %d points : %d" % (Five_Features_Test.shape[0],(Target_Array_Test != y_pred_5).sum()))

Number of mislabeled points out of a total 9824 points : 5310


In [89]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_5_svm = clf.fit(Five_Features, Target_Array).predict(Five_Features_Test)
y_pred_list_svm_5 = y_pred_5_svm.tolist()
%store y_pred_list_svm_5 > y_pred_list_svm_5.txt

Writing 'y_pred_list_svm_5' (list) to file 'y_pred_list_svm_5.txt'.


In [90]:
print("Number of mislabeled points out of a total %d points : %d" % (Five_Features_Test.shape[0],(Target_Array_Test != y_pred_5_svm).sum()))

Number of mislabeled points out of a total 9824 points : 5284


In [91]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_5_lr = clf.fit(Five_Features, Target_Array).predict(Five_Features_Test)
y_pred_list_lr_5 = y_pred_5_lr.tolist()
%store y_pred_list_lr_5 > y_pred_list_lr_5.txt

Writing 'y_pred_list_lr_5' (list) to file 'y_pred_list_lr_5.txt'.


In [92]:
print("Number of mislabeled points out of a total %d points : %d" % (Five_Features_Test.shape[0],(Target_Array_Test != y_pred_5_lr).sum()))

Number of mislabeled points out of a total 9824 points : 5078


In [93]:
Four_Features_H = np.concatenate((Three_Features, Hypothesis_Sentiment_Array), axis=1)

In [94]:
Four_Features_Test_H = np.concatenate((Three_Features_Test, Hypothesis_Sentiment_Array_Test), axis=1)

In [96]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_4_h = gnb.fit(Four_Features_H, Target_Array).predict(Four_Features_Test_H)
y_pred_4_list_h = y_pred_4_h.tolist()
%store y_pred_4_list_h > y_pred_4_list_h.txt

Writing 'y_pred_4_list_h' (list) to file 'y_pred_4_list_h.txt'.


In [97]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_H.shape[0],(Target_Array_Test != y_pred_4_h).sum()))

Number of mislabeled points out of a total 9824 points : 5025


In [102]:
print 9824 - 5025

4799


In [103]:
print 4799./9824.

0.488497557003


In [98]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_4_svm_h = clf.fit(Four_Features_H, Target_Array).predict(Four_Features_Test_H)
y_pred_list_svm_4_h = y_pred_4_svm_h.tolist()
%store y_pred_list_svm_4_h > y_pred_list_svm_4_h.txt

Writing 'y_pred_list_svm_4_h' (list) to file 'y_pred_list_svm_4_h.txt'.


In [99]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_H.shape[0],(Target_Array_Test != y_pred_4_svm_h).sum()))

Number of mislabeled points out of a total 9824 points : 5373


In [100]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_4_lr_h = clf.fit(Four_Features_H, Target_Array).predict(Four_Features_Test_H)
y_pred_list_lr_4_h = y_pred_4_lr_h.tolist()
%store y_pred_list_lr_4_h > y_pred_list_lr_4_h.txt

Writing 'y_pred_list_lr_4_h' (list) to file 'y_pred_list_lr_4_h.txt'.


In [101]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_H.shape[0],(Target_Array_Test != y_pred_4_lr_h).sum()))

Number of mislabeled points out of a total 9824 points : 5085


In [104]:
Four_Features_M = np.concatenate((Three_Features, Match_Array), axis=1)

In [105]:
Four_Features_Test_M = np.concatenate((Three_Features_Test, Match_Array_Test), axis=1)

In [106]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_4_m = gnb.fit(Four_Features_H, Target_Array).predict(Four_Features_Test_H)
y_pred_4_list_m = y_pred_4_m.tolist()
%store y_pred_4_list_m > y_pred_4_list_m.txt

Writing 'y_pred_4_list_m' (list) to file 'y_pred_4_list_m.txt'.


In [107]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_M.shape[0],(Target_Array_Test != y_pred_4_m).sum()))

Number of mislabeled points out of a total 9824 points : 5025


In [108]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_4_svm_m = clf.fit(Four_Features_M, Target_Array).predict(Four_Features_Test_M)
y_pred_list_svm_4_m = y_pred_4_svm_m.tolist()
%store y_pred_list_svm_4_m > y_pred_list_svm_4_m.txt

Writing 'y_pred_list_svm_4_m' (list) to file 'y_pred_list_svm_4_m.txt'.


In [109]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_M.shape[0],(Target_Array_Test != y_pred_4_svm_m).sum()))

Number of mislabeled points out of a total 9824 points : 5358


In [110]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_4_lr_m = clf.fit(Four_Features_M, Target_Array).predict(Four_Features_Test_M)
y_pred_list_lr_4_m = y_pred_4_lr_m.tolist()
%store y_pred_list_lr_4_m > y_pred_list_lr_4_m.txt

Writing 'y_pred_list_lr_4_m' (list) to file 'y_pred_list_lr_4_m.txt'.


In [111]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_H.shape[0],(Target_Array_Test != y_pred_4_lr_m).sum()))

Number of mislabeled points out of a total 9824 points : 5232


In [69]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
ps = PorterStemmer()
wordCount = defaultdict(int)
for j in range(len(pp.data)):
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    for w in hypothesis:
        wordCount[w]+=1
    



In [70]:
%store wordCount > wordCount.txt

Writing 'wordCount' (defaultdict) to file 'wordCount.txt'.


In [71]:
# Limit the vocabulary
wordCount = eval(open("wordCount.txt").read())
uni_sorted = sorted(wordCount.items(), key=operator.itemgetter(1)) [::-1]
vocab = [item[0] for item in uni_sorted[:10000]]
print vocab[:100]

[u'man', u'peopl', u'woman', u'two', u'play', u'girl', u'boy', u'dog', u'men', u'sit', u'wear', u'person', u'walk', u'stand', u'group', u'women', u'young', u'child', u'outside.', u'ride', u'look', u'hold', u'run', u'three', u'children', u'watch', u'kid', u'outsid', u'eat', u'take', u'get', u'work', u'black', u'near', u'wait', u'jump', u'player', u'white', u'ladi', u'front', u'coupl', u'red', u'one', u'go', u'blue', u'guy', u'dress', u'perform', u'littl', u'shirt', u'sleep', u'street.', u'make', u'bike', u'old', u'street', u'talk', u'tri', u'water.', u'worker', u'swim', u'outdoors.', u'beach.', u'next', u'someon', u'car', u'crowd', u'water', u'pictur', u'insid', u'use', u'around', u'park.', u'danc', u'enjoy', u'friend', u'human', u'drink', u'babi', u'larg', u'read', u'four', u'ball', u'small', u'game', u'build', u'park', u'carri', u'cat', u'soccer', u'game.', u'green', u'race', u'together.', u'drive', u'pose', u'climb', u'anim', u'famili', u'prepar']


In [72]:
print len(vocab)

10000


In [73]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Char_Vector_List = []
for j in range(len(pp.data)):
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    L = []
    for w in hypothesis:
        if w in vocab:
            index = vocab.index(w)
            L.append(index)
    Char_Vector_List.append(L)                    

In [74]:
%store Char_Vector_List > Char_Vector_List.txt

Writing 'Char_Vector_List' (list) to file 'Char_Vector_List.txt'.


In [98]:
print Char_Vector_List[0]

[11, 148, 108, 306]


In [105]:
print len(Char_Vector_List)

549367


In [75]:
from scipy.sparse import csr_matrix
Char_Vector_List = eval(open("Char_Vector_List.txt").read())
indptr = [0]
length = 0
indices = []
for d in Char_Vector_List:
    length = length + len(d)
    indptr.append(length)
    for j in range(len(d)):
        indices.append(d[j])
data = np.ones(length)        
X = csr_matrix((data, indices, indptr), dtype=int)

In [76]:
print X.shape

(549367, 10000)


In [77]:
Three_Features_Sparse = csr_matrix(Three_Features)

In [78]:
from scipy.sparse import hstack
Many_Features = hstack([Three_Features_Sparse,X])

In [79]:
print Many_Features.shape

(549367, 10003)


In [80]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Char_Vector_Array_Test = np.zeros((len(tt.data),len(vocab)))
for j in range(len(tt.data)):
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    L = []
    for w in hypothesis:
        if w in vocab:
            index = vocab.index(w)
            Char_Vector_Array_Test[j,index] = 1       

In [81]:
print Char_Vector_Array_Test.shape

(9824, 10000)


In [82]:
Many_Features_Test = np.concatenate((Three_Features_Test, Char_Vector_Array_Test), axis=1)

In [83]:
print Many_Features_Test.shape

(9824, 10003)


In [84]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_many_lr_train = clf.fit(Many_Features, Target_Array).predict(Many_Features)
y_pred_list_many_lr_train = y_pred_many_lr_train.tolist()
%store y_pred_list_many_lr_train > y_pred_list_many_lr_train.txt

Writing 'y_pred_list_many_lr_train' (list) to file 'y_pred_list_many_lr_train.txt'.


In [85]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features.shape[0],(Target_Array != y_pred_many_lr_train).sum()))

Number of mislabeled points out of a total 549367 points : 184547


In [86]:
print 549367 - 184547

364820


In [87]:
print 364820./549367.

0.664073378998


In [95]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_many_lr = clf.fit(Many_Features, Target_Array).predict(Many_Features_Test)
y_pred_list_many_lr = y_pred_many_lr.tolist()
%store y_pred_list_many_lr > y_pred_list_many_lr.txt

Writing 'y_pred_list_many_lr' (list) to file 'y_pred_list_many_lr.txt'.


In [166]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features_Test.shape[0],(Target_Array_Test != y_pred_many_lr).sum()))

Number of mislabeled points out of a total 9824 points : 3301


In [167]:
print 9824 - 3301

6523


In [168]:
print 6523./9824

0.663986156352


In [96]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_many_lr))

             precision    recall  f1-score   support

contradiction       0.65      0.67      0.66      3237
 entailment       0.67      0.74      0.70      3368
    neutral       0.67      0.58      0.62      3219

avg / total       0.66      0.66      0.66      9824



In [97]:
metrics.confusion_matrix(Target_Array_Test, y_pred_many_lr)

array([[2166,  573,  498],
       [ 464, 2479,  425],
       [ 684,  657, 1878]])

In [88]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_many_svm_train = clf.fit(Many_Features, Target_Array).predict(Many_Features)
y_pred_list_many_svm_train = y_pred_many_svm_train.tolist()
%store y_pred_list_many_svm_train > y_pred_list_many_svm_train.txt

Writing 'y_pred_list_many_svm_train' (list) to file 'y_pred_list_many_svm_train.txt'.


In [89]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features.shape[0],(Target_Array != y_pred_many_svm_train).sum()))

Number of mislabeled points out of a total 549367 points : 203710


In [90]:
print 549367 - 203710

345657


In [91]:
print 345657./549367.

0.629191414847


In [169]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_many_svm = clf.fit(Many_Features, Target_Array).predict(Many_Features_Test)
y_pred_list_many_svm = y_pred_many_svm.tolist()
%store y_pred_list_many_svm > y_pred_list_many_svm.txt

Writing 'y_pred_list_many_svm' (list) to file 'y_pred_list_many_svm.txt'.


In [170]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features_Test.shape[0],(Target_Array_Test != y_pred_many_svm).sum()))

Number of mislabeled points out of a total 9824 points : 3838


In [92]:
print 9824 - 3838

5986


In [93]:
print 5986./9824.

0.609324104235
